# Data Cleaning

### Configuración

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from repo_funciones import *


### Adecuación archivo censo

Buscamos homogenizar los valores de la columna fuentes para reducir el numero de opciones y poder agruparlos. 
Empezamos con la premisa de que si tu fuente no es del servicio publico (i.e agua corriente) pasas a disponibilidad no.
Vemos hay muchos valores unicos que se pueden agrupar como por ejemplo 'de una pipa' y 'la trae una pipa'.
Esto nos dará visuales mas interesantesen BI.

In [28]:
censo=pd.read_csv('../DATA/censo_clean.csv', encoding='latin-1')

In [29]:
type(censo)

pandas.core.frame.DataFrame

In [30]:
col_unique_counts(censo, 'fuente')

,fuente,Count
0,No especificado,32
1,Del servicio público de agua,16
2,De un pozo comunitario,16
3,De un pozo particular,16
4,De una pipa,16
5,De otra vivienda,16
6,De la lluvia,16
7,De otro lugar,16
8,Acarreada de un pozo,16
9,Acarreada de llave comunitaria,16


In [40]:
censo.loc[censo['fuente']=='De una pipa' , 'fuente']='Camion cisterna'
censo.loc[censo['fuente']=='La trae una pipa' , 'fuente']='Camion cisterna'
censo.loc[censo['fuente']=='Acarreada de otra vivienda' , 'fuente']='De otra vivienda'
censo.loc[censo['fuente']=='La recolectan de la lluvia' , 'fuente']='De la lluvia'
censo.loc[censo['fuente']=='De otro lugar' , 'fuente']='No especificado'
censo.loc[censo['fuente']=='Acarreada de un pozo' , 'fuente']='De un pozo comunitario'
censo.loc[censo['fuente']=='Acarreada de llave comunitaria' , 'fuente']='Llave o pozo comunitario'
censo.loc[censo['fuente']=='De un pozo comunitario' , 'fuente']='Llave o pozo comunitario'

In [47]:
censo.loc[censo['fuente']=='De una pipa' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='Camion cisterna' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='De otra vivienda' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='De la lluvia' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='No especificado' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='Llave o pozo comunitario' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='De un pozo particular' , 'disponibilidad']='no'

In [49]:
censo_group=censo.groupby(['alcaldia','disponibilidad', 'fuente'])['poblacion'].sum()


In [50]:
censo_grouped=pd.DataFrame(censo_group)

In [53]:
censo_grouped.head(10)

poblacion
alcaldia      disponibilidad fuente                                 
Azcapotzalco  no             Camion cisterna                    2345
                             De la lluvia                         15
                             De otra vivienda                    293
                             De un pozo particular                65
                             Llave o pozo comunitario            860
                             No especificado                     525
              si             Del servicio público de agua     426697
Benito Juárez no             Camion cisterna                     360
                             De la lluvia                          8
                             De otra vivienda                     67

In [54]:
censo_grouped.to_csv('../DATA/censo_grouped.csv')

### Adecuación archivo captación

Tomamos el archivo de proyectos de captación de lluvias del proyecto ETL. Recopila los proyectos de 2022. 
A este primero le tendremos que añadir una columna de año ya que le vamos a añadir los archivos respectivos de 19 a 21. 
Esto porque nos interesa medir la evolución de proyectos/capacidad/alacladías a través del tiempo. 

De igual forma tendremos que igualar el orden de las columnas para que cuadren a la hora de concatenar. 

In [4]:
captacion_22=pd.read_csv('../DATA/CLEAN/captacion_clean.csv')

In [6]:
captacion_22['año']=2022

In [7]:
captacion_22.head()

,capacidad,alcaldia,latitud,longitud,alcaldia_id,año
0,2500,Milpa Alta,19.219028,-98.991283,9,2022
1,2500,Milpa Alta,19.219880,-99.002227,9,2022
2,2500,Milpa Alta,19.218343,-98.997507,9,2022
3,2500,Milpa Alta,19.219277,-98.996269,9,2022
4,2500,Milpa Alta,19.222071,-98.997760,9,2022


In [17]:
col_unique_counts(captacion_22, 'alcaldia')

,alcaldia,Count
0,Milpa Alta,8750
1,Tlalpan,3016
2,Tláhuac,2373
3,Iztapalapa,1360
4,Gustavo A. Madero,916
5,Xochimilco,556


In [44]:
#Borramos columna capacidad, ya que tras hechar un ojo a los archivos de años pasados, niguno trae esta información.
captacion_22.drop('capacidad', axis=1, inplace=True)


In [9]:
captacion_19=pd.read_csv('../DATA/RAW/scall_2019.csv')

In [12]:
captacion_19.info() #vemos que este trae la indo de alcaldía bien llenada no como el de 2022 por lo que nos ahorra tener que limpiar. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10003 entries, 0 to 10002
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   long        10003 non-null  float64
 1   lat         10003 non-null  float64
 2   Expediente  10003 non-null  object 
 3   Pueblo      10003 non-null  object 
 4   Colonia_Ba  10003 non-null  object 
 5   Territoria  10003 non-null  object 
 6   Alcaldia    10003 non-null  object 
 7   Fecha_de_I  10000 non-null  object 
 8   Instalado   10003 non-null  object 
dtypes: float64(2), object(7)
memory usage: 703.5+ KB


In [19]:
captacion_19['Alcaldia'] = captacion_19['Alcaldia'].str.title()


In [22]:
def map_to_alcaldia_id(df, col_name):
    alcaldia_dict = {
        'Azcapotzalco': '002',
        'Coyoacán': '003',
        'Cuajimalpa de Morelos': '004',
        'Gustavo A. Madero': '005',
        'Iztacalco': '006',
        'Iztapalapa': '007',
        'La Magdalena Contreras': '008',
        'Milpa Alta': '009',
        'Álvaro Obregón': '010',
        'Tláhuac': '011',
        'Tlalpan': '012',
        'Xochimilco': '013',
        'Benito Juárez': '014',
        'Cuauhtémoc': '015',
        'Miguel Hidalgo': '016',
        'Venustiano Carranza': '017'
    }
    
    df['alcaldia_id'] = df[col_name].map(alcaldia_dict)
    
    return df.head()


In [23]:
map_to_alcaldia_id(captacion_19, 'Alcaldia')

,long,lat,Expediente,Pueblo,Colonia_Ba,Territoria,Alcaldia,Fecha_de_I,Instalado,alcaldia_id
0,-98.975686,19.328924,J16NRB,TEOTONGO ACAHUALTEPEC,CAMPESTRE POTRERO,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-07-24,Programa SCALL,007
1,-98.970402,19.327016,J5WU9F,TEOTONGO ACAHUALTEPEC,CAMPESTRE POTRERO,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-07-24,Programa SCALL,007
2,-98.969184,19.326854,X4S7RV,TEOTONGO ACAHUALTEPEC,SAN FRANCISCO APOLOCALCO,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-07-23,Programa SCALL,007
3,-98.995963,19.334764,F5AXVR,TEOTONGO ACAHUALTEPEC,MIRAVALLES,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-07-25,Programa SCALL,007
4,-99.006770,19.353941,M8IJD1,TEOTONGO ACAHUALTEPEC,SANTIAGO ACAHUALTEPEC 1a amp,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-06-06,Programa SCALL,007


In [28]:
col_unique_counts(captacion_19, 'Alcaldia')

,Alcaldia,Count
0,Iztapalapa,5833
1,Xochimilco,4170


In [31]:
col_unique_counts(captacion_19, 'alcaldia_id')
#para checar cuadra con los valores unicos de arriba y que la formula custom funcionó bien

,alcaldia_id,Count
0,007,5833
1,013,4170


In [34]:
captacion_19.drop(columns=['Expediente', 'Colonia_Ba', 'Territoria', 'Fecha_de_I', 'Instalado', 'Pueblo'], inplace=True)

In [35]:
captacion_19['año']=2019

In [37]:
captacion_19 = captacion_19.rename(columns={'Alcaldia': 'alcaldia'})

In [39]:
captacion_19 = captacion_19.rename(columns={'long': 'longitud'})
captacion_19 = captacion_19.rename(columns={'lat': 'latitud'})

In [40]:
captacion_19.head()

,longitud,latitud,alcaldia,alcaldia_id,año
0,-98.975686,19.328924,Iztapalapa,007,2019
1,-98.970402,19.327016,Iztapalapa,007,2019
2,-98.969184,19.326854,Iztapalapa,007,2019
3,-98.995963,19.334764,Iztapalapa,007,2019
4,-99.006770,19.353941,Iztapalapa,007,2019


In [41]:
captacion_19 = captacion_19.reindex(columns=['alcaldia', 'longitud', 'latitud', 'alcaldia_id', 'año'])

In [42]:
captacion_19.head()

,alcaldia,longitud,latitud,alcaldia_id,año
0,Iztapalapa,-98.975686,19.328924,007,2019
1,Iztapalapa,-98.970402,19.327016,007,2019
2,Iztapalapa,-98.969184,19.326854,007,2019
3,Iztapalapa,-98.995963,19.334764,007,2019
4,Iztapalapa,-99.006770,19.353941,007,2019


In [45]:
#19 listo, proximo año

In [46]:
captacion_20=pd.read_csv('../DATA/RAW/scall_2020.csv')

In [47]:
captacion_20.head()

,long,lat,Expediente,Pueblo,Colonia_Ba,Territoria,Alcaldia,Fecha_de_I,Instalado
0,-99.101716,19.348128,3-VESAJO-IZA-20,NO APLICA,VALLE DE LUCES,LOS CULHUACANES,IZTAPALAPA,2020-08-18,Programa SCALL
1,-99.101154,19.375799,11-ARCAKA-IZA-20,NO APLICA,SAN JOSE ACULCO,ACULCO,IZTAPALAPA,2020-10-03,Programa SCALL
2,-99.051573,19.319776,20-PAMAMA-IZA-20,NO APLICA,EL TRIANGULO,SANTA CRUZ QUETZALCOATL,IZTAPALAPA,2020-11-18,Programa SCALL
3,-99.071098,19.374533,23-COLOLU-IZA-20,NO APLICA,LEYES DE REFORMA 3ERA SECCION,LEYES DE REFORMA,IZTAPALAPA,2020-09-25,Programa SCALL
4,-99.025270,19.326348,30-SASONO-IZA-20,NO APLICA,BUENAVISTA,SANTA CATARINA,IZTAPALAPA,2020-10-02,Programa SCALL


In [48]:
captacion_20['Alcaldia'] = captacion_20['Alcaldia'].str.title()
map_to_alcaldia_id(captacion_20, 'Alcaldia')
captacion_20.drop(columns=['Expediente', 'Colonia_Ba', 'Territoria', 'Fecha_de_I', 'Instalado', 'Pueblo'], inplace=True)
captacion_20['año']=2020
captacion_20 = captacion_20.rename(columns={'Alcaldia': 'alcaldia'})
captacion_20 = captacion_20.rename(columns={'long': 'longitud'})
captacion_20 = captacion_20.rename(columns={'lat': 'latitud'})
captacion_20 = captacion_20.reindex(columns=['alcaldia', 'longitud', 'latitud', 'alcaldia_id', 'año'])

In [53]:
captacion_20.head()

,alcaldia,longitud,latitud,alcaldia_id,año
0,Iztapalapa,-99.101716,19.348128,007,2020
1,Iztapalapa,-99.101154,19.375799,007,2020
2,Iztapalapa,-99.051573,19.319776,007,2020
3,Iztapalapa,-99.071098,19.374533,007,2020
4,Iztapalapa,-99.025270,19.326348,007,2020


In [55]:
captacion_21=pd.read_csv('../DATA/RAW/scall_2021.csv')

In [56]:
captacion_21.head()

,long,lat,Expediente,Pueblo,Colonia_Ba,Territoria,Alcaldia,Fecha_de_I,Instalado
0,-99.186038,19.493667,2-MAMUNO-AZO-21,NO APLICA,PUEBLO SAN ANDRES,NO APLICA,AZCAPOTZALCO,2021-05-25,Programa SCALL
1,-99.219528,19.475549,4-MASAHO-AZO-21,NO APLICA,AMPL. SAN PEDRO XALPA,NO APLICA,AZCAPOTZALCO,2021-09-02,Programa SCALL
2,-99.211189,19.480327,5-LOLOMA-AZO-21,NO APLICA,PUEBLO SANTIAGO AHUIZOTLA,NO APLICA,AZCAPOTZALCO,2021-05-10,Programa SCALL
3,-99.176603,19.486284,6-SAARJO-AZO-21,NO APLICA,SANTO TOMAS,NO APLICA,AZCAPOTZALCO,2021-05-27,Programa SCALL
4,-99.218356,19.481075,12-PEALAR-AZO-21,NO APLICA,AMPL. SAN PEDRO XALPA,NO APLICA,AZCAPOTZALCO,2021-08-25,Programa SCALL


In [57]:
captacion_21['Alcaldia'] = captacion_21['Alcaldia'].str.title()
map_to_alcaldia_id(captacion_21, 'Alcaldia')
captacion_21.drop(columns=['Expediente', 'Colonia_Ba', 'Territoria', 'Fecha_de_I', 'Instalado', 'Pueblo'], inplace=True)
captacion_21['año']=2020
captacion_21 = captacion_21.rename(columns={'Alcaldia': 'alcaldia'})
captacion_21 = captacion_21.rename(columns={'long': 'longitud'})
captacion_21 = captacion_21.rename(columns={'lat': 'latitud'})
captacion_21 = captacion_21.reindex(columns=['alcaldia', 'longitud', 'latitud', 'alcaldia_id', 'año'])

In [58]:
captacion_21.head()

,alcaldia,longitud,latitud,alcaldia_id,año
0,Azcapotzalco,-99.186038,19.493667,002,2020
1,Azcapotzalco,-99.219528,19.475549,002,2020
2,Azcapotzalco,-99.211189,19.480327,002,2020
3,Azcapotzalco,-99.176603,19.486284,002,2020
4,Azcapotzalco,-99.218356,19.481075,002,2020


In [60]:
#exportamos...luego hacemos concat
captacion_21.to_csv('../DATA/CLEAN/captacion21_clean.csv')
captacion_22.to_csv('../DATA/CLEAN/captacion22_clean.csv')
captacion_20.to_csv('../DATA/CLEAN/captacion20_clean.csv')
captacion_19.to_csv('../DATA/CLEAN/captacion19_clean.csv')